YBIGTA 10기 노혜미 박승리

# RNN을 이용한 Text Classification with tensorflow

본격적으로 들어가기에 잠깐...!<br>
딥러닝을 이용해서 텍스트를 분류하는데는 사실 RNN도 쓰이고 CNN도 쓰인다!<BR>
그럼 궁금한 점이 생길 것이다...<BR> RNN과 CNN은 각각 어떤 텍스트를 분류할 때 좋은 걸까?<br>
일단 텍스트는 **순서**를 가지는 자료이기 때문에 RNN이 좀 더 자연스러운 접근 방법이다. 하지만 RNN은 느린 편이고 train하기에 불안정하다고 한다. (출처 글에 그렇게 나왔다.) CNN이 RNN보다 훨씬 계산 속도가 빠르다고 한다.<BR>
이와 같이 RNN에서 어느 정도의 단점이 있더라도, 하려는 일이 long semantics이냐 feature detection이냐에 따라서 적합한 모델이 다르다고 한다.<BR>
텍스트의 길이가 중요한 작업에서는 RNN을 쓰는 게 좋다고 한다. RNN은 이전의 자료에 대한 정보가 쭉 이어지기 때문이다.뭐 예를 들면, 질문하고 답하기, 번역 같은 곳에서 쓰이면 좋다고 한다.<BR>
feature detection이 더 중요하다면 CNN이 더 좋다고 한다. 감정 찾기, 개체명(namedn entity) 같은 걸 찾는 작업이 이에 해당할 것이다.

좀 더 자세하게 알고 싶다면 다음의 논문을 참고하길 바란다.<br>
[ComparativeStudyofCNNandRNNforNaturalLanguageProcessing](https://arxiv.org/pdf/1702.01923...)

In [1]:
import argparse

import sys

import numpy as np

import pandas as pd

from sklearn import metrics

import tensorflow as tf

# Data loading

[★data는 여기에★](https://www.kaggle.com/samdeeplearning/deepnlp)

In [2]:
data = pd.read_csv('Sheet_1.csv')

In [3]:
data.head()

response_id        class                                      response_text  \
0  response_1  not_flagged              I try and avoid this sort of conflict   
1  response_2      flagged  Had a friend open up to me about his mental ad...   
2  response_3      flagged  I saved a girl from suicide once. She was goin...   
3  response_4  not_flagged  i cant think of one really...i think i may hav...   
4  response_5  not_flagged  Only really one friend who doesn't fit into th...   

  Unnamed: 3  Unnamed: 4 Unnamed: 5  Unnamed: 6 Unnamed: 7  
0        NaN         NaN        NaN         NaN        NaN  
1        NaN         NaN        NaN         NaN        NaN  
2        NaN         NaN        NaN         NaN        NaN  
3        NaN         NaN        NaN         NaN        NaN  
4                    NaN        NaN         NaN        NaN

In [3]:
x = data['response_text']
y = [1 if x == 'flagged' else 0 for x in data['class'] ] # flagged -> 1, not flagged -> 0

# Model & Detail about model

아래 소스는 다음의 github를 보고 썼다.

소스 코드 출처:<br>
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/learn/text_classification.py

약간 지나칠 정도로 설명을 해놨으므로 가독성이 떨어질 수도 있다ㅠㅠ<br>
부제(?)를 나름 크게 해놨으니까 아는거다 싶으면 휙휙 넘어가면 된다!

### embedding size는 얼마로 할까?
- word2vec 같은 neural network에서는 보통 크기를 300~500으로 셋팅한다고 한다.<br>
 하지만, 자신의 data가 매우 적고 토이 모델 수준이라면 굳이 많이 할 필요가 없다.<br>
 embedding size가 클수록 vector의 차원이 늘어나므로 계산량이 확 늘어나기 때문이다.<br>
 오랜 시간이 걸릴지라도 최고의 결과를 내고싶다! 이런 사람들은 500정도 주면 되지 않을까...?

In [10]:
FLAGS = None

MAX_DOCUMENT_LENGTH = 25 # 문서의 최대 길이. 길이가 1~25사이에 몰려 있어서 이렇게 했다.

EMBEDDING_SIZE = 20 # 단어를 벡터로 바꿔주는 것이 (Word) embedding 이다. 이 embedding의 크기를 얼마로 해주는지 설정해준다. 
                    # 많지 않은 데이터의 경우 차원을 크게 해주면 오히려 공간에서 흩어지므로 좋지 않을 것 같다.
n_words = 0

MAX_LABEL = 2 # 0 or 1

WORDS_FEATURE = 'words'  # Name of the input words feature. feature naming에 사용되므로, 큰 의미가 있는 것 같지는 않다.(아마도...?)

learning_rate = 0.01

## estimator
- train, test, evaluation mode를 한번에 지정해주는 단계이다.

#### 함수의 특징을 잘 모른다면...!
- 보통 파이썬 문법에서 여러 가지의 케이스가 있을 때는 if, elif, else와 같이 케이스를 나누는데 아래의 코드에서는 if만 쓰던가 혹은 아무것도 쓰지 않는다. 그 이유는 굳이 쓸 필요가 없기 때문이다. <br> 함수 내부에서 return을 만나면 이후에 코드가 남았든 반복문 안이든 간에 프로그램은 함수를 빠져나온다. 즉, 함수가 필요한 값을 돌려주고 끝난다는 소리이다. <br>아래의 코드를 예로 들자면, Mode가 PREDICT인 경우 이후에 있는 그 다음 if문 부터의 코드는 수행되지 않는다.

#### tf.estimator.EstimatorSpec
- estimator specification의 약자같다.<br> 메서드를 정의 해놓고 이후의 코드에서 해당 메서드에 mode값만 다르게 주면 알아서 train, test, evaluation을 해주기 위해서 만들어진 듯 싶다.<br> 실행 부분에서 코드의 간결성을 주는 메서드 같다.

#### tf.one_hot
- one_hot(indices, depth, on_value=None, off_value=None, axis=None, dtype=None, name=None)<br>
 depth만큼의 크기를 가지는 vector에서, indices에 해당하는 index값에 on_value를 주고 그 외의 값에는 off_value를 주겠다는 뜻이다.<br>
 해당 코드에서는 2열 짜리 vector에서 lable에 해당하는 index값에 1을 그 외의 값을 0으로 하겠다는 뜻이다. (사실 1과 0은 default값)

In [5]:
def estimator_spec_for_softmax_classification(logits, labels, mode):

    "Returns EstimatorSpec instance for softmax classification."

    predicted_classes = tf.argmax(logits, 1)
        
    # case 1) inference mode
    
    # ModeKey가 PREDICT이면 inference mode이다. inference는 학습시킨 모델을 새로운 데이터에 적용시키는 것을 말한다.
    if mode == tf.estimator.ModeKeys.PREDICT: 

        return tf.estimator.EstimatorSpec(mode=mode, predictions={'class': predicted_classes, 'prob': tf.nn.softmax(logits)})

    
    # case 2) training mode
    
    onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0) 

    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)    
    
    if mode == tf.estimator.ModeKeys.TRAIN:

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


    # case 3) evaluation mode
    
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## rnn_model
- input 데이터를 embedding 시켜주고, neural network를 구성하는 단계이다. 마지막 결과를 softmax를 통해 0~1의 값으로 반환하는 단계이다.

#### tf.contrib.layers.embed_sequence

- symbols의 순서를 embeddings의 순서로 mapping 해주는 method.<br>
  embedded sequences를 가지는 [batch_size, doc_length, embed_dim]를 반환한다.<br>
  embed_dim만큼의 크기를 가지는 벡터들이 document 혹은 sentence의 word 개수만큼 있고, <br> 
  document 혹은 sentence를 몇 개씩 짝 지어놓은 게 batch_size만큼 있다.
  
#### word_vectors & word_list
![이해](http://i.imgur.com/Bjn2LRF.png)

#### tf.layers.dense

- outputs = activation(inputs.kernel + bias)의 연산을 수행한다. 만약 activation 함수를 주면 activation 함수의 input으로 해당 값이 들어간다.<br>
 kernel은 weight matrix이다.

In [6]:
def rnn_model(features, labels, mode):

    """RNN model to predict from sequence of words to a class."""
    
    # 단어들의 index를 embeddings로 바꾼다. 
    # [n_words(단어 개수), EMBEDDING_SIZE]의 크기를 가지는 matrix를 만들고 순서를 나타내는 단어들의 index를 
    # [batch_size, sequence_length, EMBEDDING_SIZE]에 mapping 한다.
    
    # word_vectors와 word_list를 만든다.
    word_vectors = tf.contrib.layers.embed_sequence(features[WORDS_FEATURE], vocab_size=n_words, embed_dim=EMBEDDING_SIZE)
    word_list = tf.unstack(word_vectors, axis=1)

    
    # embedding size와 같은 hidden size를 가지는 GUR cell을 만든다.
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)

    
    # MAX_DOCUMENT_LENGTH의 길이를 가지는 RNN을 만든다. 그리고 각 유닛에 input으로 word_list를 준다.
    # (output, state)쌍이 return된다.
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)

    # 마지막 유닛의 state 값을 softmax classification의 feature로 넘겨준다.
    logits = tf.layers.dense(encoding, MAX_LABEL, activation=None)

    return estimator_spec_for_softmax_classification(logits=logits, labels=labels, mode=mode)

In [7]:
from sklearn.model_selection import train_test_split

## main

- 이전에 정의한 다른 estimator, rnn_model과 같은 함수들을 불러와 실행시켜주는 단계이다.

#### tf.contrib.learn.preprocessing.VocabularyProcessor

- 문장의 길이를 우리가 원하는 sequence length로 맞추어 주는 method.<br>
  문장을 길이가 MAX_DOCUMENT_LENGTH인 vector로 반환한다.<br>
  만약 MAX_DOCUMENT_LENGTH보다 작다면 0 pad가 더해지고, 크다면 이후 값은 쓰지 않는다.<br>
  Embedding 이전 단계의 데이터 형태이다.<br>
  
  ![rnn link](http://i.imgur.com/IDaLPgj.png)


#### tf.estimator.Estimator

- 지정해주는 모델을 적용하는 classifier를 만들어준다.<br>
  Train, Test, Predict를 모두 실행시켜준다.<br>

In [13]:
def main(unused_argv):
    global n_words
    
    # train data set, test data set을 나눠준다.
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 4)
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test) # tensorflow input으로 사용될 수 있는 자료형으로 변환시켜줘야 한다.
    
    # 단어들을 우리가 원하는 sequence length로 맞추어 준다.(embedding 해주기 전 단계) 위의 사진 참고
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    
    x_transform_train = vocab_processor.fit_transform(x_train) # 둘의 차이는?
    x_transform_test = vocab_processor.transform(x_test)
    
    x_train = np.array(list(x_transform_train))
    x_test = np.array(list(x_transform_test))
    
    n_words = len(vocab_processor.vocabulary_)
    print('Total words : %d', n_words)
    
    # 모델을 만들어준다.(여기서는 위에서 정의한 rnn_model을 사용한다.)
    model_fn = rnn_model
    classifier = tf.estimator.Estimator(model_fn=model_fn)
    
    # Train
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {WORDS_FEATURE : x_train},
        y = y_train,
        batch_size = len(x_train),
        num_epochs = None,
        shuffle = True) # shuffle = True : 
    classifier.train(input_fn = train_input_fn, steps = 100)
    
    # Predict
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {WORDS_FEATURE : x_test},
        y = y_test,
        num_epochs = 1,
        shuffle = False)
    predictions = classifier.predict(input_fn = test_input_fn)
    y_predicted = np.array(list(p['class'] for p in predictions))
    
    # Score using tensorflow
    score = classifier.evaluate(input_fn = test_input_fn)
    print('Accuracy (tensorflow): {0:f}'.format(score['accuracy']))

## 실행단계
- 위에서 bow model(bag of word model(수요일에 진행))은 정의하지 않았으므로, 위에서 정의한 rnn_model을 사용하게 된다.

In [14]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--test_with_fake_data',
        default = False,
        help = 'Test the example code with fake data',
        action = 'store_true')
    parser.add_argument(
        '--bow model',
        default = False,
        help = 'Run with BOW model instead of RNN',
        action = 'store_true')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv = [sys.argv[0]] + unparsed)

Total words : %d 650
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\nhm\\AppData\\Local\\Temp\\tmpzp_gogxa', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\nhm\AppData\Local\Temp\tmpzp_gogxa\model.ckpt.
INFO:tensorflow:loss = 0.73507, step = 1
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\nhm\AppData\Local\Temp\tmpzp_gogxa\model.ckpt.
INFO:tensorflow:Loss for final step: 5.99285e-05.
INFO:tensorflow:Restoring parameters from C:\Users\nhm\AppData\Local\Temp\tmpzp_gogxa\model.ckpt-100
INFO:tensorflow:Starting evaluation at 2017-08-11-16:28:07
INFO:tensorflow:Restoring parameters from C:\Users\nhm\AppData\Local\Temp\tmpzp_gogxa\model.ckpt-100
INFO:tensorflow:Finished evaluatio

SystemExit: 

C:\Users\nhm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 마무리!

- Neural Network는 black box라고 한다. 결과가 나왔을 때, 왜 이런 결과가 나왔는지 모른다는 것이 딥러닝의 가장 큰 단점이 아닐까 싶다. <br>
  위의 accuracy가 마음에 들지 않아서 hyper parameter를 조정해주면서 반복해보았지만 그렇게 큰 차이가 나지는 않았다. <br>
  우리는 GRUCell을 사용했는데, 어쩌면 LSTM이 더 좋은 결과를 가져다 줄 지도 모르겠다. <br>
  
  
- 이전까지는 단순한 모델이였기에 함수를 정의하지 않고, 순차적으로 딥러닝을 구현했다. 처음에 NN의 각 역할을 함수로 나눠서 정의한 후에, <br>
  나중에 한번에 합친다는 것 자체가 어색했지만(지금까지 함수를 정의한 경험이 많이 없다.), 구현해보고 나니 왜 함수로 나누는지 알 수 있었다. <br>
  
- 마지막에 실행단계에서 error가 처음에 발생했는데, train, test data를 나눌 때 sklearn을 사용한 결과는 list여서 input으로 사용될 수 없었던 <br>
  문제였다. 함수별로 나눈 덕에 빠르게 오류의 원인을 찾아낼 수 있었다. 또한 전체적인 구조가 눈에 확 들어와서 좋은 방법이라고 느낄 수 있었다.

## Reference

- https://www.quora.com/Which-is-better-for-text-classification-CNN-or-RNN-Which-areas-of-NLP-do-they-better-suit-to
- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/learn/text_classification.py
- https://medium.com/@ilblackdragon/tensorflow-text-classification-615198df9231
- https://stackoverflow.com/questions/40661684/tensorflow-vocabularyprocessor
- https://github.com/hunkim/word-rnn-tensorflow/blob/master/model.py